In [36]:
# from googletrans import Translator
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import pandas as pd
import pyspark
from pyspark.sql.functions import split
from pyspark.sql.functions import explode, col, trim, rtrim, ltrim
from pyspark.sql.functions import regexp_replace, regexp_extract
from pyspark.sql.functions import lower
from pyspark.sql.functions import desc
from pyspark.ml.feature import StopWordsRemover
import re
import glob
import os

In [33]:
def count_words(txt_path):
    #pyspark session
    spark = SparkSession.builder.appName("PySpark-ContandoPalavras").getOrCreate()
    sc = spark.sparkContext

    #txt doc
    df = spark.read.text(f"{txt_path}")

    #spliting the df by space " " 
    df = df.select(
    split(df['value'], ' ').alias('words'))

    #removing stopwords
    stopwords_remover = StopWordsRemover(inputCol="words", outputCol="words2")
    df = stopwords_remover.transform(df)
    df = df.drop("words")

    #extracting words from the lists
    df = df.select(
    explode(col("words2")).alias("words3"))
    
    # Removing the NULLs
    df = df.where(
        col("words3") != "")
    
    #removing punctuation
    df = df.select(
    regexp_extract(
    col('words3'),  #cleaning words2 column
    '[A-z]+',       #selecting all the words
    0).alias('words4'))

    #every word to lower case
    df = df.select(
    lower(
    col('words4')
    ).alias('words5'))

    #FIRST COUNT#
    df_counted = df.groupBy(
        col("words5"),
    ).count()

    #RETAKING
    df2 = df.select(
        split(
            col("words5"),
            " "
        ).alias("word6"))
    
    #removing stopwords - step2
    stopwords_remover2 = StopWordsRemover(
    inputCol="word6",
    outputCol="words7")
    df2 = stopwords_remover2.transform(df2)

    #removing nulls - step2
    df2 = df2.select(
        (col("words7")[0]).alias("words8"))

    df2 = df2.where(
    col("words8") != "null")

    df2 = df2.where(
        col("words8") != "")
    
    #FINAL COUNT#
    df_counted2 = df2.groupby(
    col("words8")
    ).count()

    df_counted2 = df_counted2.orderBy(desc("count")).toPandas()
    
    return df_counted2
    # return df_counted2.orderBy(desc("count")).show()

In [35]:
#folder path
folder_path = 'C:/Users/cedua/CDIA - PUCSP/PROJETO PySpark - SAVINO/Contador-de-palavras---PySpark/data'

#iterating the txt files only
for txt_file in glob.glob(os.path.join(folder_path, '*.txt')):
    df_pandas = count_words(txt_file)
    df_pandas.to_csv(f'{txt_file}.csv', sep=';', index=False)

<class 'pandas.core.frame.DataFrame'>
